In [1]:
import logging
import os

# set the directory to the location of the script
try:
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

In [2]:
from src.redis.redis_client import AzureRedisManager

In [3]:
redis_manager = AzureRedisManager()

redis_manager.ping()  # Check connectivity
redis_manager.store_session_data(
    "session123", {"user": "Pablo", "authenticated": "True"}
)
print(redis_manager.get_session_data("session123"))

2025-05-19 22:57:55,442 - micro - MainProcess - INFO     Azure Redis connection initialized. (redis_client.py:__init__:42)
INFO:micro:Azure Redis connection initialized.


{'user': 'Pablo', 'authenticated': 'True'}


In [5]:
from src.speech.speech_to_text import SpeechCoreTranslator  # existing class

In [ ]:
speech.create_realtime_recognizer

In [2]:
speechsdk.PropertyId.Speech_SegmentationStrategy

NameError: name 'speechsdk' is not defined

In [3]:
import os
from typing import Callable, Optional, Tuple

import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.speech import SpeechRecognitionResult
from dotenv import load_dotenv

from utils.ml_logging import get_logger
import asyncio

# Set up logger
logger = get_logger()

# Load environment variables from .env file
load_dotenv()

True

In [4]:
speechsdk.PropertyId.Speech_SegmentationStrategy

<PropertyId.Speech_SegmentationStrategy: 9004>

In [13]:
speech_key = os.getenv("AZURE_SPEECH_KEY")
speech_region = os.getenv("AZURE_SPEECH_REGION")
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.set_property(
    speechsdk.PropertyId.SpeechServiceConnection_EnableAudioLogging, "true"
)

In [ ]:
# recognizer.properties.set_property(
#     speechsdk.PropertyId.Speech_SegmentationSilenceTimeoutMs,
#     str(50),
# )
# recognizer.properties.set_property(
#     speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, str(vad_silence_timeout_ms))

In [14]:
speech_config.set_property(
    speechsdk.PropertyId.Speech_SegmentationSilenceTimeoutMs, str(50)
)
speech_config.set_property(
    speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, str(50)
)

In [1]:
from azure.cognitiveservices.speech.audio import (
    AudioStreamFormat,
    PushAudioInputStream,
    AudioProcessingOptions,
    AudioProcessingConstants,
)

ImportError: cannot import name 'AudioProcessingOptions' from 'azure.cognitiveservices.speech.audio' (c:\Users\pablosal\AppData\Local\anaconda3\envs\audioagent\Lib\site-packages\azure\cognitiveservices\speech\audio.py)

In [ ]:
audio_config = speechsdk.audio.AudioConfig(stream=push_stream)

# Use the instance's speech_config
recognizer = speechsdk.SpeechRecognizer(
    speech_config=self.speech_config,
    audio_config=audio_config,
    language=language,
)

In [8]:
class SpeechCoreTranslator:
    """
    A class that serves as the core for handling Azure AI Services Speech SDK functionality.
    It encapsulates the processes involved in translating and transcribing speech.
    """

    def __init__(self):
        self.speech_key = os.getenv("AZURE_SPEECH_KEY")
        self.speech_region = os.getenv("AZURE_SPEECH_REGION")
        self.connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
        self.speech_config = speechsdk.SpeechConfig(
            subscription=self.speech_key, region=self.speech_region
        )
        self.speech_config.set_property(
            speechsdk.PropertyId.SpeechServiceConnection_EnableAudioLogging, "true"
        )
        self.supported_languages = [
            "en-US",  # English (United States)
            "es-ES",  # Spanish (Spain)
            "fr-FR",  # French (France)
        ]

    def add_supported_language(self, language):
        """
        Appends a language to the list of supported languages.
        Parameters:
        language (str): The language to be added.
        """
        self.supported_languages.append(language)

    def create_realtime_recognizer(
        self,
        push_stream: speechsdk.audio.PushAudioInputStream,
        loop: asyncio.AbstractEventLoop,
        message_queue: asyncio.Queue,
        language: str = "en-US",
        vad_silence_timeout_ms: int = 1000,
    ):
        """
        Creates and configures a SpeechRecognizer for real-time continuous recognition
        using a PushAudioInputStream and sets up callbacks to use an asyncio Queue.

        Args:
            push_stream: The PushAudioInputStream to use for audio input.
            loop: The asyncio event loop for scheduling callbacks.
            message_queue: The asyncio Queue to put recognized text onto.
            language: The language code for recognition (e.g., "en-US").

        Returns:
            A configured speechsdk.SpeechRecognizer instance.
        """
        audio_config = speechsdk.audio.AudioConfig(stream=push_stream)

        # Use the instance's speech_config
        recognizer = speechsdk.SpeechRecognizer(
            speech_config=self.speech_config,
            audio_config=audio_config,
            language=language,
        )

        # recognizer.properties.set_property(
        #     speechsdk.PropertyId.Speech_SegmentationSilenceTimeoutMs,
        #     str(50),
        # )
        # recognizer.properties.set_property(
        #     speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, str(vad_silence_timeout_ms))

        # --- Define Callbacks ---
        def recognizing_cb(evt: speechsdk.SpeechRecognitionEventArgs):
            if evt.result.reason == speechsdk.ResultReason.RecognizingSpeech:
                logger.info(f"PARTIAL → {evt.result.text}")

        def recognized_cb(evt: speechsdk.SpeechRecognitionEventArgs):
            if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
                logger.info(f"FINAL   → {evt.result.text}")
                # Put the final recognized text onto the queue for processing
                if evt.result.text:  # Avoid putting empty strings on the queue
                    asyncio.run_coroutine_threadsafe(
                        message_queue.put(evt.result.text), loop
                    )
            elif evt.result.reason == speechsdk.ResultReason.NoMatch:
                logger.info("NOMATCH: Speech could not be recognized.")

        def session_stopped_cb(evt: speechsdk.SessionEventArgs):
            logger.info(f"Session stopped: {evt}")
            # You might want to signal the end of the stream or handle cleanup here
            # Example: asyncio.run_coroutine_threadsafe(message_queue.put(None), loop)

        def canceled_cb(evt: speechsdk.SpeechRecognitionCanceledEventArgs):
            logger.error(f"Recognition Canceled: {evt.reason}")
            if evt.reason == speechsdk.CancellationReason.Error:
                logger.error(f"Canceled details: {evt.error_details}")
            # Signal the end or error state
            # Example: asyncio.run_coroutine_threadsafe(message_queue.put(None), loop)

        # Connect callbacks
        recognizer.recognizing.connect(recognizing_cb)
        recognizer.recognized.connect(recognized_cb)
        recognizer.session_stopped.connect(session_stopped_cb)
        recognizer.canceled.connect(canceled_cb)

        logger.info(f"Realtime recognizer created for language: {language}")
        return recognizer

In [9]:
speach = SpeechCoreTranslator()

In [11]:
speach.create_realtime_recognizer(
    push_stream=None,
    loop=asyncio.get_event_loop(),
    message_queue=asyncio.Queue(),
    language="en-US",
)

ValueError: cannot construct AudioConfig with the given arguments

In [12]:
from azure.cognitiveservices.speech.audio import (
    AudioProcessingOptions,
    AudioProcessingConstants,
)

ImportError: cannot import name 'AudioProcessingOptions' from 'azure.cognitiveservices.speech.audio' (c:\Users\pablosal\AppData\Local\anaconda3\envs\audioagent\Lib\site-packages\azure\cognitiveservices\speech\audio.py)

In [5]:
import os

import azure.cognitiveservices.speech as speechsdk
from dotenv import load_dotenv

from utils.ml_logging import get_logger

# Load environment variables from a .env file if present
load_dotenv()

# Initialize logger
logger = get_logger()


class SpeechSynthesizer:
    def __init__(
        self,
        key: str = None,
        region: str = None,
        language: str = "en-US",
        voice: str = "en-US-JennyMultilingualNeural",
    ):
        # Retrieve Azure Speech credentials from parameters or environment variables
        self.key = key or os.getenv("AZURE_SPEECH_KEY")
        self.region = region or os.getenv("AZURE_SPEECH_REGION")
        self.language = language
        self.voice = voice

        # Initialize the speech synthesizer for speaker playback
        self.speaker_synthesizer = self._create_speaker_synthesizer()

    def _create_speech_config(self):
        """
        Helper method to create and configure the SpeechConfig object.
        """
        speech_config = speechsdk.SpeechConfig(
            subscription=self.key, region=self.region
        )
        speech_config.speech_synthesis_language = self.language
        speech_config.speech_synthesis_voice_name = self.voice
        # Set the output format to 24kHz 16-bit mono PCM WAV
        speech_config.set_speech_synthesis_output_format(
            speechsdk.SpeechSynthesisOutputFormat.Riff24Khz16BitMonoPcm
        )
        return speech_config

    def _create_speaker_synthesizer(self):
        """
        Create a SpeechSynthesizer instance for playing audio through the server's default speaker.
        """
        speech_config = self._create_speech_config()
        audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
        return speechsdk.SpeechSynthesizer(
            speech_config=speech_config, audio_config=audio_config
        )

    def start_speaking_text(self, text: str) -> None:
        """
        Asynchronously play synthesized speech through the server's default speaker.
        """
        try:
            logger.info(f"[🔊] Speaking text (server speaker): {text[:30]}...")
            self.speaker_synthesizer.start_speaking_text_async(text)
        except Exception as e:
            logger.error(f"[❗] Error starting speech synthesis: {e}")

    def stop_speaking(self) -> None:
        """
        Stop any ongoing speech synthesis playback on the server's speaker.
        """
        try:
            logger.info("[🛑] Stopping speech synthesis on server speaker...")
            self.speaker_synthesizer.stop_speaking_async()
        except Exception as e:
            logger.error(f"[❗] Error stopping speech synthesis: {e}")

    def synthesize_speech(self, text: str) -> bytes:
        """
        Synthesizes text to speech in memory (returning WAV bytes).
        Does NOT play audio on server speakers.
        """
        try:
            speech_config = speechsdk.SpeechConfig(
                subscription=self.key, region=self.region
            )
            speech_config.speech_synthesis_language = self.language
            speech_config.speech_synthesis_voice_name = self.voice
            speech_config.set_speech_synthesis_output_format(
                speechsdk.SpeechSynthesisOutputFormat.Riff48Khz16BitMonoPcm
            )

            synthesizer = speechsdk.SpeechSynthesizer(
                speech_config=speech_config, audio_config=None
            )

            result = synthesizer.speak_text_async(text).get()

            if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                audio_data_stream = speechsdk.AudioDataStream(result)
                wav_bytes = audio_data_stream.read_data()  # ✅ USE read_data()
                return bytes(
                    wav_bytes
                )  # ✅ Ensure it's converted from bytearray to bytes
            else:
                logger.error(f"Speech synthesis failed: {result.reason}")
                return b""
        except Exception as e:
            logger.error(f"Error synthesizing speech: {e}")
            return b""

    def synthesize_to_base64_frames(
        self, text: str, sample_rate: int = 16000
    ) -> list[str]:
        """
        Synthesize `text` via Azure TTS into raw 16-bit PCM mono at either 16 kHz or 24 kHz,
        then split into 20 ms frames (50 fps), returning each frame as a base64 string.

        - sample_rate: 16000 or 24000
        - frame_size:  0.02s * sample_rate * 2 bytes/sample
                    =  640 bytes @16 kHz, 960 bytes @24 kHz
        """
        # Select SDK output format and packet size
        fmt_map = {
            16000: speechsdk.SpeechSynthesisOutputFormat.Raw16Khz16BitMonoPcm,
            24000: speechsdk.SpeechSynthesisOutputFormat.Raw24Khz16BitMonoPcm,
        }
        sdk_format = fmt_map.get(sample_rate)
        if not sdk_format:
            raise ValueError("sample_rate must be 16000 or 24000")

        # 1) Configure Speech SDK using class attributes
        speech_config = speechsdk.SpeechConfig(
            subscription=self.key, region=self.region
        )
        speech_config.speech_synthesis_language = self.language
        speech_config.speech_synthesis_voice_name = self.voice
        speech_config.set_speech_synthesis_output_format(sdk_format)

        # 2) Synthesize to memory (audio_config=None)
        synth = speechsdk.SpeechSynthesizer(
            speech_config=speech_config, audio_config=None
        )

        # 3) Build an SSML envelope with reduced rate (80%)
        ##  If you would like to speed up the speech, you can increase the `prosody rate`% accordingly.

        ssml = f"""
        <speak version="1.0" xmlns:mstts="http://www.w3.org/2001/mstts" xml:lang="en-US">
          <voice name="{speech_config.speech_synthesis_voice_name}">
            <mstts:express-as style="chat">
              <prosody rate="85%" pitch="default">
                {text}
              </prosody>
            </mstts:express-as>
          </voice>
        </speak>
        """
        # 4) Synthesize
        result = synth.speak_ssml_async(ssml).get()
        if result.reason != speechsdk.ResultReason.SynthesizingAudioCompleted:
            error_details = result.cancellation_details
            logger.error(f"TTS failed: {result.reason}")
            if error_details:
                logger.error(f"Error details: {error_details.error_details}")
                logger.error(f"Error code: {error_details.error_code}")
            raise RuntimeError(f"TTS failed: {result.reason}")

        # 5) Get raw PCM bytes from the result
        pcm_bytes = result.audio_data  # Access audio data directly from the result

        return bytes(pcm_bytes)  # Ensure it's bytes type

In [6]:
speechtest = SpeechSynthesizer()

In [7]:
speechtest.synthesize_to_base64_frames("Hi There", 16000)

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x